# Function Calling

* ref: [OpenAI의 함수 호출(Function Calling) 소개 및 Python 예시](https://techbukket.com/blog/openai-function-python)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from openai import OpenAI
client = OpenAI()

## 0. 호출할 함수 정의

In [3]:
import json

def get_current_weather(location, unit="섭씨"):
    """
    지정된 위치의 현재 날씨 정보를 반환하는 함수.

    이 함수는 입력받은 위치에 대한 날씨 정보를 포함하는 딕셔너리를 생성하고,
    이를 JSON 형식의 문자열로 변환하여 반환합니다. 반환되는 날씨 정보에는
    위치, 기온, 단위(섭씨/화씨), 예보(맑음, 바람 등)가 포함됩니다.

    Args:
        location (str): 날씨 정보를 얻고자 하는 위치의 이름.
        unit (str, optional): 온도 표시 단위. 기본값은 "섭씨". "화씨"로 변경 가능.

    Returns:
        str: 지정된 위치의 날씨 정보를 포함하는 JSON 형식의 문자열.
    """
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info, ensure_ascii=False)

## 1. 모델에 함수 정보와 메시지를 전달

In [4]:
messages = [{"role": "user", "content": "오늘 서울 날씨 어때??"}]

In [5]:
functions = [
    {
        "name": "get_current_weather",
        "description": "주어진 지역의 현재 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "지역, e.g. 서울, 부산, 제주도",
                },
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
            },
            "required": ["location"],
        },
    }
]

In [9]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    functions=functions,
    function_call="auto"
)

print("첫번재 응답")
print(response)

response_message = response.choices[0].message

첫번재 응답
ChatCompletion(id='chatcmpl-8cXPhViyrPiC9TCE2yh91vbBg5shQ', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"서울"}', name='get_current_weather'), tool_calls=None))], created=1704195597, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=17, prompt_tokens=114, total_tokens=131))


## 2. GPT 응답의 함수를 실행

In [7]:
function_response = get_current_weather("서울")
print(function_response)

{"location": "서울", "temperature": "24", "unit": "섭씨", "forecast": ["sunny", "windy"]}


## 3. 모델에 함수를 실행한 결과를 전달

In [8]:
function_name="get_current_weather"

In [10]:
messages.append(response_message) # messages에 GPT의 첫번째 응답을 추가합니다.
messages.append( # messages에 함수의 실행결과를 추가합니다.
    {
        "role": "function",
        "name": function_name, # 함수 이름
        "content": function_response, # 함수 실행 결과
    }
)

In [13]:
# 두번째 OPENAI API 요청
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
)

# 응답을 출력
print(second_response)

ChatCompletion(id='chatcmpl-8cXQUXkWywIZ3CjgBU9geALjaAiFN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='서울의 오늘 날씨는 24도로 맑고 바람이 조금 불고 있습니다.', role='assistant', function_call=None, tool_calls=None))], created=1704195646, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=33, prompt_tokens=84, total_tokens=117))
